In [1]:
import wandb

In [2]:
from sam2.build_sam import build_sam2_video_predictor
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import os

from sam2.build_sam import build_sam2_video_predictor_npz

In [3]:
from omegaconf import OmegaConf
from training.train import single_node_runner
from training.utils.train_utils import register_omegaconf_resolvers





In [10]:
pwd

'/gpfs/data/arsoyd01lab/machlm03/Segmentation/MedSAM2'

In [4]:
register_omegaconf_resolvers()
cfg_path = "/gpfs/home/machlm03/Segmentation/MedSAM2/sam2/configs/sam2.1_hiera_tiny512_FLARE_RECIST.yaml"
cfg = OmegaConf.load(cfg_path)


available_gpus = torch.cuda.device_count()
if "device" in cfg.trainer.model:
    dev = cfg.trainer.model.device
    if isinstance(dev, str) and dev.startswith("cuda:"):
        idx = int(dev.split(":")[1])
        if idx >= available_gpus:
            cfg.trainer.model.device = "cuda:0"  # fallback

In [5]:
# cfg.trainer.data

In [6]:
available_gpus = torch.cuda.device_count()

cfg.launcher.gpus_per_node = 1
cfg.launcher.num_nodes = 1

# cfg.launcher.gpus_per_node = min(cfg.launcher.gpus_per_node, available_gpus)

if cfg.launcher.gpus_per_node == 1:
    print("⚠️ Only one GPU detected — forcing single-process mode")


⚠️ Only one GPU detected — forcing single-process mode


In [7]:
import wandb

wandb.init(
    project="medsam2-finetune",
    name="fold1",
    config=OmegaConf.to_container(cfg, resolve=True)
)


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: naseem-machlovi (machlovi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
# print(cfg.output_dir)
# print(cfg.trainer)  # if present



In [9]:
from training.train import single_proc_run
# cfg.trainer.data.train.batch_sizes = [1]

# make sure config says single GPU
cfg.launcher.gpus_per_node = 1
cfg.launcher.num_nodes = 1
world_size = 1
main_port = 12355

single_proc_run(
    local_rank=0,
    main_port=main_port,
    cfg=cfg,
    world_size=world_size,
    node_rank=0,
    master_addr="127.0.0.1",
)


INFO 2025-08-22 12:12:10,665 train_utils.py: 108: MACHINE SEED: 1230
INFO 2025-08-22 12:12:10,698 train_utils.py: 154: Logging ENV_VARIABLES
INFO 2025-08-22 12:12:10,698 train_utils.py: 155: ANACONDA3_ROOT=/gpfs/share/apps/anaconda3/gpu/2023.09
BASH_ENV=/cm/local/apps/environment-modules/4.4.0//init/bash
BASH_FUNC_create_passwd%%=() {  tr -cd 'a-zA-Z0-9' < /dev/urandom 2> /dev/null | head -c${1:-8}
}
BASH_FUNC_find_port%%=() {  local host="${1:-localhost}";
 local port=$(random_number "${2:-2000}" "${3:-65535}");
 while port_used "${host}:${port}"; do
 port=$(random_number "${2:-2000}" "${3:-65535}");
 done;
 echo "${port}"
}
BASH_FUNC_module%%=() {  unset _mlshdbg;
 if [ "${MODULES_SILENT_SHELL_DEBUG:-0}" = '1' ]; then
 case "$-" in 
 *v*x*)
 set +vx;
 _mlshdbg='vx'
 ;;
 *v*)
 set +v;
 _mlshdbg='v'
 ;;
 *x*)
 set +x;
 _mlshdbg='x'
 ;;
 *)
 _mlshdbg=''
 ;;
 esac;
 fi;
 unset _mlre _mlIFS;
 if [ -n "${IFS+x}" ]; then
 _mlIFS=$IFS;
 fi;
 IFS=' ';
 for _mlv in ${MODULES_RUN_QUARANTINE:-};

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


INFO 2025-08-22 12:12:12,524 sam2.py:  81: Training with points (sampled from masks) as inputs with p=0.5
INFO 2025-08-22 12:12:12,526 trainer.py:1106: ====================
INFO 2025-08-22 12:12:12,526 trainer.py:1107: Summary for model <class 'training.model.sam2.SAM2Train'>
INFO 2025-08-22 12:12:12,528 trainer.py:1108: Model is SAM2Train(
  (image_encoder): ImageEncoder(
    (trunk): Hiera(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      )
      (blocks): ModuleList(
        (0): MultiScaleBlock(
          (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (attn): MultiScaleAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (proj): Linear(in_features=96, out_features=96, bias=True)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): MLP(
            (layers): ModuleLis

/gpfs/home/machlm03/.conda/envs/medsam2/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:4631: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
/gpfs/home/machlm03/.conda/envs/medsam2/lib/python3.12/site-packages/torch/autograd/graph.py:824: UserWarning: Grad strides do not match bucket view strides. This may indicate grad was not created according to the gradient layout contract, or that the param's strides changed since DDP was constructed.  This is not an error, but may impair performance.
grad.sizes() = [64, 256, 1, 1], strides() = [256, 1, 256, 256]
bucket_view.sizes() = [64, 256, 1, 1], strides() = [256, 1, 1, 1] (Triggered internally at /pytorch/torch/csrc/distributed/c10d/reducer.cpp:328.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


INFO 2025-08-22 12:12:34,967 train_utils.py: 271: Train Epoch: [0][0/5] | Batch Time: 18.39 (18.39) | Data Time: 15.50 (15.50) | Mem (GB): 55.00 (55.00/55.00) | Time Elapsed: 00d 00h 00m | Losses/train_all_loss: 1.71e+00 (1.71e+00) | Losses/train_vos_loss: 0.00e+00 (0.00e+00)
INFO 2025-08-22 12:13:01,085 trainer.py: 983: Estimated time remaining: 00d 00h 06m
INFO 2025-08-22 12:13:01,086 trainer.py: 926: Synchronizing meters
INFO 2025-08-22 12:13:01,086 trainer.py: 867: Losses and meters: {'Losses/train_all_loss': 3.6105692625045775, 'Losses/train_vos_loss': 0, 'Losses/train_all_loss_mask': 0.09328365568071603, 'Losses/train_all_loss_dice': 1.0642080664634705, 'Losses/train_all_loss_iou': 0.3223196566104889, 'Losses/train_all_loss_class': 0.35836826839949937, 'Losses/train_all_core_loss': 3.6105692625045775, 'Trainer/where': 0.08, 'Trainer/epoch': 0, 'Trainer/steps_train': 5}


...

INFO 2025-08-22 12:15:44,311 trainer.py: 983: Estimated time remaining: 00d 00h 09m
INFO 2025-08-22 12:15:44,312 trainer.py: 926: Synchronizing meters
INFO 2025-08-22 12:15:44,312 trainer.py: 867: Losses and meters: {'Losses/train_all_loss': 3.1727493166923524, 'Losses/train_vos_loss': 0, 'Losses/train_all_loss_mask': 0.05264227855950594, 'Losses/train_all_loss_dice': 1.4970196425914764, 'Losses/train_all_loss_iou': 0.6070266216993332, 'Losses/train_all_loss_class': 0.015857644472271205, 'Losses/train_all_core_loss': 3.1727493166923524, 'Trainer/where': 0.27999999999999997, 'Trainer/epoch': 2, 'Trainer/steps_train': 15}


wandb: WARNING Tried to log to step 15 that is less than the current step 20. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


INFO 2025-08-22 12:15:57,815 train_utils.py: 271: Val Epoch: [2][0/5] | Batch Time: 12.89 (12.89) | Data Time: 12.32 (12.32) | Mem (GB): 3.00 (3.00/3.00) | Time Elapsed: 00d 00h 03m | Losses/val_all_loss: 0.00e+00 (0.00e+00) | Losses/val_vos_loss: 4.19e+00 (4.19e+00)
INFO 2025-08-22 12:16:21,568 trainer.py: 983: Estimated time remaining: 00d 00h 09m
INFO 2025-08-22 12:16:21,569 trainer.py: 926: Synchronizing meters
INFO 2025-08-22 12:16:21,569 trainer.py: 729: Meters: {'Losses/val_all_loss': 0, 'Losses/val_vos_loss': 3.5390583992004396, 'Losses/val_vos_loss_mask': 0.04660669341683388, 'Losses/val_vos_loss_dice': 1.9703646183013916, 'Losses/val_vos_loss_iou': 0.6365588426589965, 'Losses/val_vos_loss_class': 1.1070115817801706e-06, 'Losses/val_vos_core_loss': 3.5390583992004396, 'Trainer/where': 0.27999999999999997, 'Trainer/epoch': 2, 'Trainer/steps_val': 30}
INFO 2025-08-22 12:16:21,763 trainer.py: 580: New best val loss: 3.539058


wandb: WARNING Tried to log to step 2 that is less than the current step 30. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


KeyboardInterrupt: 

In [ ]:
# cfg.trainer.data.val.sampler

In [13]:
cfg.trainer.data.train.datasets[0].dataset.datasets[0].video_dataset.file_list_txt

'/gpfs/home/machlm03/Segmentation/OAI_demo/OAI_TrainTest/cv_txt_files/train/train_fold0.txt'

In [12]:


## to run k fold

from omegaconf import OmegaConf
from training.train import single_proc_run

NUM_FOLDS = 5
main_port = 12355

for fold in range(NUM_FOLDS):
    print(f"🔁 Starting Fold {fold}")

    # Load base config
    # Load base config
    train_file = f"/gpfs/home/machlm03/Segmentation/OAI_demo/OAI_TrainTest/cv_txt_files/train/train_fold{fold}.txt"
    val_file = f"/gpfs/home/machlm03/Segmentation/OAI_demo/OAI_TrainTest/cv_txt_files/train/train_fold{fold}.txt"

    # Load base config
    cfg = OmegaConf.load("/gpfs/home/machlm03/Segmentation/MedSAM2/sam2/configs/sam2.1_hiera_tiny512_FLARE_RECIST.yaml")

    # Update config for single GPU
    cfg.launcher.gpus_per_node = 1
    cfg.launcher.num_nodes = 1
    world_size = 1



    cfg.trainer.data.train.datasets[0].dataset.datasets[0].video_dataset.file_list_txt = train_file

    cfg.trainer.data.val = {
        "_target_": "training.dataset.vos_dataset.VOSDataset",
        "transforms": cfg.vos.val_transforms,
        "training": False,
        "video_dataset": {
            "_target_": "training.dataset.vos_raw_dataset.NPZRawDataset",
            "file_list_txt": val_file,
            "folder": "/gpfs/home/machlm03/Segmentation/OAI_demo/OAI_TrainTest/V00_00m_MultiClass/npz/"
        },
        "sampler": {
            "_target_": "training.dataset.vos_sampler.SequentialSampler"
        }
    }

    # Optional: tag the run with fold number
    cfg.fold_id = fold

    # Run training for this fold
    single_proc_run(
        local_rank=0,
        main_port=main_port,
        cfg=cfg,
        world_size=world_size,
        node_rank=0,
        master_addr="127.0.0.1",
    )

    print(f"✅ Finished Fold {fold}\n")
    

🔁 Starting Fold 0
INFO 2025-08-22 12:07:49,134 train.py:  38: resolver 'get_method' is already registered
INFO 2025-08-22 12:07:49,156 train_utils.py:  76: Setting up torch.distributed with a timeout of 30 mins


InstantiationException: Error in call to target 'training.trainer.Trainer':
ValueError('trying to initialize the default process group twice!')
full_key: trainer

In [ ]:
# cfg.trainer.data.train.batch_sizes = [1]

# main_port = 12355
# single_node_runner(cfg, main_port=main_port, node_rank=0)
cfg.trainer.data.val

In [ ]:
print(OmegaConf.to_container(cfg, resolve=False).keys())
print(cfg.trainer.keys())


In [ ]:
val_dataset = cfg.trainer.data.val.datasets # first vos dataset
print(val_dataset)

# from hydra.utils import instantiate

# # Instantiate the dataset object
# val_dataset_obj = instantiate(val_dataset)

# print(type(val_dataset_obj))  # should be ConcatDataset
# print(len(val_dataset_obj))   # number of samples


In [ ]:
val_dataset = cfg.trainer.data.val.datasets['vos'][0]  # first vos dataset
print(val_dataset)

In [ ]:
import os

dataset_path = "/gpfs/home/machlm03/Segmentation/OAI_demo/images/"
npz_files = [f for f in os.listdir(dataset_path) if f.endswith(".gz")]

print(f"Found {len(npz_files)} .gz files")





In [ ]:
from hydra.utils import instantiate

# Test sampler instantiation
sampler_config = {
    '_target_': 'training.dataset.vos_sampler.RandomUniformSampler',
    'num_frames': 4,  # replace with actual value
    'max_num_objects': 1  # replace with actual value
}

sampler = instantiate(sampler_config)
print(f"Sampler type: {type(sampler)}")
print(f"Has sample method: {hasattr(sampler, 'sample')}")

In [ ]:
cfg.trainer.data.train.datasets

In [ ]:
# TRAIN sampler config
train_dataset_cfg = cfg.trainer.data.train.datasets[0]['dataset']['datasets'][0]
train_sampler_cfg = train_dataset_cfg['sampler']
print("TRAIN sampler config:")
print(train_sampler_cfg)
print(f"Type: {type(train_sampler_cfg)}")

# VAL sampler config
val_dataset_cfg = cfg.trainer.data.val.datasets['vos'][0]['datasets'][0]
val_sampler_cfg = val_dataset_cfg['sampler']
print("\nVAL sampler config:")
print(val_sampler_cfg)
print(f"Type: {type(val_sampler_cfg)}")


In [ ]:
# First create the NPZRawDataset
from training.dataset.vos_dataset import VOSDataset

from training.dataset.vos_raw_dataset import NPZRawDataset

npz_dataset = NPZRawDataset(
    file_list_txt='/gpfs/home/machlm03/Segmentation/OAI_demo/OAI_TrainTest/cv_txt_files/train/val_fold0.txt',
    folder='/gpfs/home/machlm03/Segmentation/OAI_demo/OAI_TrainTest/V00_00m_MultiClass/npz/'
)

# Then test the VOSDataset with it
vos_dataset = VOSDataset(
    transforms=None,
    training=False,
    video_dataset=npz_dataset,  # This is the NPZRawDataset instance
    sampler={
        '_target_': 'training.dataset.vos_sampler.RandomUniformSampler',
        'num_frames': 4,
        'max_num_objects': 1
    },
    multiplier=1
)

print(f"Sampler type after init: {type(vos_dataset.sampler)}")
print(f"Sampler value: {vos_dataset.sampler}")

# Test if we can get an item
try:
    item = vos_dataset[0]
    print(f"SUCCESS! Item type: {type(item)}")
    print(f"Has frames: {hasattr(item, 'frames')}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
from hydra.utils import instantiate
from training.dataset.vos_dataset import VOSDataset
# Test your fixed VOSDataset
vos_dataset = VOSDataset(
    transforms=None,
    training=False,
    video_dataset=npz_dataset,
    sampler={
        '_target_': 'training.dataset.vos_sampler.RandomUniformSampler',
        'num_frames': 4,
        'max_num_objects': 1
    },
    multiplier=1
)

print("Testing fixed VOSDataset:")
try:
    item = vos_dataset[0]
    print(f"SUCCESS! Item type: {type(item)}")
    print(f"Has frames: {hasattr(item, 'frames')}")
except Exception as e:
    print(f"Still error: {e}")

In [ ]:
# 1. Test NPZRawDataset directly
print("=== Testing NPZRawDataset directly ===")
try:
    raw_item = npz_dataset[0]
    print(f"Raw dataset item type: {type(raw_item)}")
    print(f"Raw dataset item: {raw_item}")
    if hasattr(raw_item, 'frames'):
        print(f"Number of frames: {len(raw_item.frames)}")
    else:
        print("No frames attribute")
except Exception as e:
    print(f"NPZRawDataset error: {e}")

# 2. Test get_video method (used by VOSDataset)
print("\n=== Testing get_video method ===")
try:
    video, segment_loader = npz_dataset.get_video(0)
    print(f"Video type: {type(video)}")
    print(f"Segment loader: {segment_loader}")
    print(f"Video has frames: {hasattr(video, 'frames')}")
except Exception as e:
    print(f"get_video error: {e}")

# 3. Test sampler separately
print("\n=== Testing sampler ===")
try:
    video, segment_loader = npz_dataset.get_video(0)
    sampled_frms_and_objs = vos_dataset.sampler.sample(
        video, segment_loader, epoch=0
    )
    print(f"Sampler output type: {type(sampled_frms_and_objs)}")
    print(f"Sampler output: {sampled_frms_and_objs}")
except Exception as e:
    print(f"Sampler error: {e}")
    

In [ ]:
# 4. Test the full VOSDataset pipeline
print("\n=== Testing VOSDataset construct method ===")
try:
    video, segment_loader = npz_dataset.get_video(0)
    sampled_frms_and_objs = vos_dataset.sampler.sample(video, segment_loader, epoch=0)
    
    # This is what happens in VOSDataset._get_datapoint()
    datapoint = vos_dataset.construct(video, sampled_frms_and_objs, segment_loader)
    print(f"Datapoint type: {type(datapoint)}")
    print(f"Datapoint: {datapoint}")
    if hasattr(datapoint, 'frames'):
        print(f"Datapoint has frames: {len(datapoint.frames)}")
    
except Exception as e:
    print(f"Construct error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
vos_dataset

In [ ]:
# Test the actual transform pipeline
print("=== Debug transform application ===")

# Get the raw datapoint before transforms
video, segment_loader = npz_dataset.get_video(0)
sampled_frms_and_objs = vos_dataset.sampler.sample(video, segment_loader, epoch=0)
raw_datapoint = vos_dataset.construct(video, sampled_frms_and_objs, segment_loader)

print(f"Before transforms - frame data type: {type(raw_datapoint.frames[0].data)}")
print(f"Number of transforms: {len(vos_dataset._transforms)}")

# Apply transforms step by step
current_datapoint = raw_datapoint
for i, transform in enumerate(vos_dataset._transforms):
    print(f"\nApplying transform {i}: {type(transform).__name__}")
    try:
        current_datapoint = transform(current_datapoint)
        if hasattr(current_datapoint, 'frames') and len(current_datapoint.frames) > 0:
            frame_data = current_datapoint.frames[0].data
            print(f"  After transform: {type(frame_data)}")
            if hasattr(frame_data, 'shape'):
                print(f"  Shape: {frame_data.shape}")
        else:
            print(f"  After transform: {current_datapoint}")
    except Exception as e:
        print(f"  Transform failed: {e}")
        import traceback
        traceback.print_exc()
        break

print(f"\nFinal datapoint type: {type(current_datapoint)}")

In [ ]:
# Test the complete VOSDataset.__getitem__() method
print("\n=== Testing complete VOSDataset[0] ===")
try:
    item = vos_dataset[0]
    print(f"SUCCESS! Final item type: {type(item)}")
    print(f"Has frames: {hasattr(item, 'frames')}")
    if hasattr(item, 'frames'):
        print(f"Number of frames: {len(item.frames)}")
        print(f"First frame data type: {type(item.frames[0].data)}")
        if hasattr(item.frames[0].data, 'shape'):
            print(f"First frame shape: {item.frames[0].data.shape}")
    
    # Test if this works with collate function
    print("\n=== Testing collate function ===")
    from training.utils.data_utils import collate_fn
    batch = [item]  # Single item batch
    
    collated = collate_fn(batch, dict_key='vos')
    print(f"Collate SUCCESS! Output type: {type(collated)}")
    
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()